In [3]:
pdf_file_path = "D:/CS6120/final_project/nlp-s25-project/data_scraper/Graduate_Catalog.pdf"

In [4]:
# from langchain_unstructured import UnstructuredLoader
# import getpass
# import os

# if "UNSTRUCTURED_API_KEY" not in os.environ:
#     os.environ["UNSTRUCTURED_API_KEY"] = getpass.getpass(
#         "A87XlsGwMwX7ZsT5DEegBXl95DF1Aa"
#     )

# loader = UnstructuredLoader(
#     file_path=pdf_file_path,
#     strategy = 'hi-res',
#     partition_via_api=True,
#     coordinates=True
# )

# docs = []
# for doc in loader.lazy_load():
#     docs.append(doc)

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader(pdf_file_path)
number_of_pages = len(reader.pages)
page = reader.pages[17]
text = page.extract_text()
print(text)
replace_string='Northeastern University 2023-2024 Graduate Catalog'
pg='17'
text.split("\n")
print(text.replace(f"{replace_string} {pg}", ""))

Northeastern University 2023-2024 Graduate Catalog 17
Information for Entering Students
Graduate education at Northeastern University integrates the highest level of scholarship across disciplinary boundaries with signiﬁcant research and
experiential learning opportunities in Boston and around the world. Northeastern offers hundreds of graduate programs, ranging from doctoral and
full-time master’s programs to part-time programs and graduate certiﬁcates, including an array of innovative PhD and master’s programs designed to
prepare students for emerging new ﬁelds. Students are able to take courses on campus, online, or in hybrid formats. This multidimensional learning
environment offers students the knowledge and experience to excel and the flexibility to create the educational experience that best meets their
needs. Our graduates are well positioned to meet the diverse demands of careers in academia, industry, and the professions.
•Academic Resources (p. 18)
•Campus Resources (p. 21)


In [25]:
big = 0
for pg in range(number_of_pages):
    length_page = len(reader.pages[pg].extract_text().split())
    if big < length_page:
        big = length_page
print(big)


1339


In [59]:
def preprocess_text(text):
    text = text.replace("\xa0"," ")
    text = text.replace("..","")
    return text

In [105]:
# we need id, title, heading, prev_heading, pg_no, 
data_catalogue = []
id=1000
title = "Graduate_Catalog"
prev_heading = ""
heading = ""
replace_string = "Northeastern University 2023-2024 Graduate Catalog"
pg = 16
chunk_length = 250
while pg<number_of_pages:
    page = reader.pages[pg]
    text = page.extract_text()
    text = preprocess_text(text)
    split_by_newline = text.split("\n")
    stretch = 1 if pg%2 == 0 else 2
    heading = " ".join(split_by_newline[:stretch]).replace(f"{replace_string} {pg}", "").replace(f"{pg}","").strip()
    rest_of_text = "\n".join(split_by_newline[stretch:]).split()
    le = len(rest_of_text)
    for i in range((le//chunk_length)+1):
        text_chunk = " ".join(rest_of_text[i*chunk_length:min((i+1)*chunk_length, le)])
        data_dict={}
        data_dict['title'] = title
        data_dict['pg_no'] = pg
        data_dict['heading'] = heading
        data_dict['prev_heading'] = prev_heading
        data_dict['id'] = f"id_{id}"
        data_dict['chunk'] = text_chunk
        data_catalogue.append(data_dict)
        id+=1
    prev_heading = heading
    pg+=1

In [110]:
import chromadb
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="husky_bot_docs")

In [111]:
collection.add(
    documents=[data['chunk'] for data in data_catalogue],
    metadatas=[{
        'page':data['pg_no'], 
        'degree':data['title'],
        'course':data['heading'],
        'alt_course':data['prev_heading']
    } for data in data_catalogue],
    ids=[data['id'] for data in data_catalogue]
)

C:\Users\surya\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:17<00:00, 4.74MiB/s]
